# Laboratorio 6

# Analisis de Redes Sociales

### Cristina Bautista 161260
### Andy Castillo 18040
### Marco Fuentes 18188
### Abril Palencia 18198

## Problema 4

### Descarga, carga - Cristina y Andy

### Preprosecamiento - Cristina

### Analisis - Marco y Abril

### Conclusiones - Andy

In [1]:
import numpy as np
import pandas as pd

In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier
from nltk import sent_tokenize, word_tokenize, pos_tag
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk import FreqDist
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/cristinabautista/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/cristinabautista/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
from wordcloud import WordCloud, STOPWORDS

import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter
%matplotlib inline

In [4]:
data = pd.read_csv('problema5_tweets.csv')

In [5]:
data.head()

,Tweet Id,Text,Name,Screen Name,UTC,Created At,Favorites,Retweets,Language,Client,Tweet Type,URLs,Hashtags,Mentions,Media Type,Media URLs
0,1433971287811108869,@prensa_libre 👎👎🥚🥚🤮🤮💩💩🐀🐀🐀🐀pira corrupción #Ren...,Aura Perez,AuraPer75486154,2021-09-04T01:52:51.000Z,Sat Sep 04 01:52:51 +0000 2021,0,0,es,"<a href=""http://twitter.com/download/android"" ...",Reply,NaN,2,1,NaN,NaN
1,1433971126389153794,@Factor4_GT @EmilianoCS @EFEnoticias @opsoms #...,Aura Perez,AuraPer75486154,2021-09-04T01:52:13.000Z,Sat Sep 04 01:52:13 +0000 2021,0,0,und,"<a href=""http://twitter.com/download/android"" ...",Reply,NaN,3,4,NaN,NaN
2,1433970899800240131,@AlianzaRgt 👎👎💩💩🥚🥚🤮🤮🤮🤮💩💩#Renuncia desconsuelo ...,Aura Perez,AuraPer75486154,2021-09-04T01:51:19.000Z,Sat Sep 04 01:51:19 +0000 2021,0,0,es,"<a href=""http://twitter.com/download/android"" ...",Reply,NaN,2,1,NaN,NaN
3,1433965107751620610,@el_Periodico Que desgracia desgobierno maldit...,Aura Perez,AuraPer75486154,2021-09-04T01:28:18.000Z,Sat Sep 04 01:28:18 +0000 2021,0,0,es,"<a href=""http://twitter.com/download/android"" ...",Reply,NaN,4,1,NaN,NaN
4,1433964253531226114,@prensa_libre #RenunciaGiamattei #RenunciaCons...,Aura Perez,AuraPer75486154,2021-09-04T01:24:54.000Z,Sat Sep 04 01:24:54 +0000 2021,0,0,und,"<a href=""http://twitter.com/download/android"" ...",Reply,NaN,3,1,NaN,NaN


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3049 entries, 0 to 3048
Data columns (total 16 columns):
Tweet Id       3049 non-null int64
Text           3049 non-null object
Name           3049 non-null object
Screen Name    3049 non-null object
UTC            3049 non-null object
Created At     3049 non-null object
Favorites      3049 non-null int64
Retweets       3049 non-null int64
Language       3049 non-null object
Client         3049 non-null object
Tweet Type     3049 non-null object
URLs           416 non-null object
Hashtags       3049 non-null int64
Mentions       3049 non-null int64
Media Type     1362 non-null object
Media URLs     1362 non-null object
dtypes: int64(5), object(11)
memory usage: 381.2+ KB


In [7]:
data = data.rename(columns={"Tweet Id":"tweetId", "Text":"text", "Name":"name", "Screen Name":"screenName",
                           "UTC":"utc", "Created At":"createdAt", "Favorites":"favorites", "Retweets":"retweets",
                           "Language":"language", "Client":"client", "Tweet Type":"tweetType", "URLs":"urls",
                           "Hashtags":"hashtags", "Mentions":"mentions", "Media Type":"mediaType", 
                           "Media URLs":"mediaURLs"})

In [8]:
data.head()

,tweetId,text,name,screenName,utc,createdAt,favorites,retweets,language,client,tweetType,urls,hashtags,mentions,mediaType,mediaURLs
0,1433971287811108869,@prensa_libre 👎👎🥚🥚🤮🤮💩💩🐀🐀🐀🐀pira corrupción #Ren...,Aura Perez,AuraPer75486154,2021-09-04T01:52:51.000Z,Sat Sep 04 01:52:51 +0000 2021,0,0,es,"<a href=""http://twitter.com/download/android"" ...",Reply,NaN,2,1,NaN,NaN
1,1433971126389153794,@Factor4_GT @EmilianoCS @EFEnoticias @opsoms #...,Aura Perez,AuraPer75486154,2021-09-04T01:52:13.000Z,Sat Sep 04 01:52:13 +0000 2021,0,0,und,"<a href=""http://twitter.com/download/android"" ...",Reply,NaN,3,4,NaN,NaN
2,1433970899800240131,@AlianzaRgt 👎👎💩💩🥚🥚🤮🤮🤮🤮💩💩#Renuncia desconsuelo ...,Aura Perez,AuraPer75486154,2021-09-04T01:51:19.000Z,Sat Sep 04 01:51:19 +0000 2021,0,0,es,"<a href=""http://twitter.com/download/android"" ...",Reply,NaN,2,1,NaN,NaN
3,1433965107751620610,@el_Periodico Que desgracia desgobierno maldit...,Aura Perez,AuraPer75486154,2021-09-04T01:28:18.000Z,Sat Sep 04 01:28:18 +0000 2021,0,0,es,"<a href=""http://twitter.com/download/android"" ...",Reply,NaN,4,1,NaN,NaN
4,1433964253531226114,@prensa_libre #RenunciaGiamattei #RenunciaCons...,Aura Perez,AuraPer75486154,2021-09-04T01:24:54.000Z,Sat Sep 04 01:24:54 +0000 2021,0,0,und,"<a href=""http://twitter.com/download/android"" ...",Reply,NaN,3,1,NaN,NaN


In [9]:
# Eliminar url
data['text'] = data['text'].str.replace(r'http\S+', ' ')
data['client'] = data['client'].str.replace(r'http\S+', ' ')
data['urls'] = data['urls'].str.replace(r'http\S+', ' ')
data['mediaURLs'] = data['mediaURLs'].str.replace(r'http\S+', ' ')

In [10]:
data.head()

,tweetId,text,name,screenName,utc,createdAt,favorites,retweets,language,client,tweetType,urls,hashtags,mentions,mediaType,mediaURLs
0,1433971287811108869,@prensa_libre 👎👎🥚🥚🤮🤮💩💩🐀🐀🐀🐀pira corrupción #Ren...,Aura Perez,AuraPer75486154,2021-09-04T01:52:51.000Z,Sat Sep 04 01:52:51 +0000 2021,0,0,es,"<a href="" rel=""nofollow"">Twitter for Android</a>",Reply,NaN,2,1,NaN,NaN
1,1433971126389153794,@Factor4_GT @EmilianoCS @EFEnoticias @opsoms #...,Aura Perez,AuraPer75486154,2021-09-04T01:52:13.000Z,Sat Sep 04 01:52:13 +0000 2021,0,0,und,"<a href="" rel=""nofollow"">Twitter for Android</a>",Reply,NaN,3,4,NaN,NaN
2,1433970899800240131,@AlianzaRgt 👎👎💩💩🥚🥚🤮🤮🤮🤮💩💩#Renuncia desconsuelo ...,Aura Perez,AuraPer75486154,2021-09-04T01:51:19.000Z,Sat Sep 04 01:51:19 +0000 2021,0,0,es,"<a href="" rel=""nofollow"">Twitter for Android</a>",Reply,NaN,2,1,NaN,NaN
3,1433965107751620610,@el_Periodico Que desgracia desgobierno maldit...,Aura Perez,AuraPer75486154,2021-09-04T01:28:18.000Z,Sat Sep 04 01:28:18 +0000 2021,0,0,es,"<a href="" rel=""nofollow"">Twitter for Android</a>",Reply,NaN,4,1,NaN,NaN
4,1433964253531226114,@prensa_libre #RenunciaGiamattei #RenunciaCons...,Aura Perez,AuraPer75486154,2021-09-04T01:24:54.000Z,Sat Sep 04 01:24:54 +0000 2021,0,0,und,"<a href="" rel=""nofollow"">Twitter for Android</a>",Reply,NaN,3,1,NaN,NaN


In [11]:
# Se elimino todo lo que no sean letras, como emojis, signos de puntuacion y numeros
data['text'] = data['text'].str.replace(r'[^A-Za-z\s]+', ' ')
data['client'] = data['client'].str.replace(r'[^A-Za-z]+', ' ')
data['tweetType'] = data['tweetType'].str.replace(r'[^A-Za-z]+', ' ')
data['mediaType'] = data['mediaType'].str.replace(r'[^A-Za-z\s]+', ' ')

In [12]:
data.head()

,tweetId,text,name,screenName,utc,createdAt,favorites,retweets,language,client,tweetType,urls,hashtags,mentions,mediaType,mediaURLs
0,1433971287811108869,prensa libre pira corrupci n RenunciaGiamat...,Aura Perez,AuraPer75486154,2021-09-04T01:52:51.000Z,Sat Sep 04 01:52:51 +0000 2021,0,0,es,a href rel nofollow Twitter for Android a,Reply,NaN,2,1,NaN,NaN
1,1433971126389153794,Factor GT EmilianoCS EFEnoticias opsoms R...,Aura Perez,AuraPer75486154,2021-09-04T01:52:13.000Z,Sat Sep 04 01:52:13 +0000 2021,0,0,und,a href rel nofollow Twitter for Android a,Reply,NaN,3,4,NaN,NaN
2,1433970899800240131,AlianzaRgt Renuncia desconsuelo RenunciaGia...,Aura Perez,AuraPer75486154,2021-09-04T01:51:19.000Z,Sat Sep 04 01:51:19 +0000 2021,0,0,es,a href rel nofollow Twitter for Android a,Reply,NaN,2,1,NaN,NaN
3,1433965107751620610,el Periodico Que desgracia desgobierno maldit...,Aura Perez,AuraPer75486154,2021-09-04T01:28:18.000Z,Sat Sep 04 01:28:18 +0000 2021,0,0,es,a href rel nofollow Twitter for Android a,Reply,NaN,4,1,NaN,NaN
4,1433964253531226114,prensa libre RenunciaGiamattei RenunciaCons...,Aura Perez,AuraPer75486154,2021-09-04T01:24:54.000Z,Sat Sep 04 01:24:54 +0000 2021,0,0,und,a href rel nofollow Twitter for Android a,Reply,NaN,3,1,NaN,NaN


In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3049 entries, 0 to 3048
Data columns (total 16 columns):
tweetId       3049 non-null int64
text          3049 non-null object
name          3049 non-null object
screenName    3049 non-null object
utc           3049 non-null object
createdAt     3049 non-null object
favorites     3049 non-null int64
retweets      3049 non-null int64
language      3049 non-null object
client        3049 non-null object
tweetType     3049 non-null object
urls          416 non-null object
hashtags      3049 non-null int64
mentions      3049 non-null int64
mediaType     1362 non-null object
mediaURLs     1362 non-null object
dtypes: int64(5), object(11)
memory usage: 381.2+ KB


In [14]:
# Tokenize
tokenizer = RegexpTokenizer(r'\w+')

data['text'] = data['text'].apply(lambda x: tokenizer.tokenize(x))
data['name'] = data['name'].apply(lambda x: tokenizer.tokenize(x))
data['language'] = data['language'].apply(lambda x: tokenizer.tokenize(x))
data['client'] = data['client'].apply(lambda x: tokenizer.tokenize(x))
data['tweetType'] = data['tweetType'].apply(lambda x: tokenizer.tokenize(x))
data['mediaType'] = data['mediaType'].apply(lambda x: tokenizer.tokenize(str(x)))

In [15]:
# Stopwords
stopwords = stopwords.words('english')

In [16]:
data['text'] = data['text'].apply(lambda y: [w for w in y if w not in stopwords])

In [17]:
data.head()

,tweetId,text,name,screenName,utc,createdAt,favorites,retweets,language,client,tweetType,urls,hashtags,mentions,mediaType,mediaURLs
0,1433971287811108869,"[prensa, libre, pira, corrupci, n, RenunciaGia...","[Aura, Perez]",AuraPer75486154,2021-09-04T01:52:51.000Z,Sat Sep 04 01:52:51 +0000 2021,0,0,[es],"[a, href, rel, nofollow, Twitter, for, Android...",[Reply],NaN,2,1,[nan],NaN
1,1433971126389153794,"[Factor, GT, EmilianoCS, EFEnoticias, opsoms, ...","[Aura, Perez]",AuraPer75486154,2021-09-04T01:52:13.000Z,Sat Sep 04 01:52:13 +0000 2021,0,0,[und],"[a, href, rel, nofollow, Twitter, for, Android...",[Reply],NaN,3,4,[nan],NaN
2,1433970899800240131,"[AlianzaRgt, Renuncia, desconsuelo, RenunciaGi...","[Aura, Perez]",AuraPer75486154,2021-09-04T01:51:19.000Z,Sat Sep 04 01:51:19 +0000 2021,0,0,[es],"[a, href, rel, nofollow, Twitter, for, Android...",[Reply],NaN,2,1,[nan],NaN
3,1433965107751620610,"[el, Periodico, Que, desgracia, desgobierno, m...","[Aura, Perez]",AuraPer75486154,2021-09-04T01:28:18.000Z,Sat Sep 04 01:28:18 +0000 2021,0,0,[es],"[a, href, rel, nofollow, Twitter, for, Android...",[Reply],NaN,4,1,[nan],NaN
4,1433964253531226114,"[prensa, libre, RenunciaGiamattei, RenunciaCon...","[Aura, Perez]",AuraPer75486154,2021-09-04T01:24:54.000Z,Sat Sep 04 01:24:54 +0000 2021,0,0,[und],"[a, href, rel, nofollow, Twitter, for, Android...",[Reply],NaN,3,1,[nan],NaN
